<a href="https://colab.research.google.com/github/VAlphonso/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-03 01:44:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-07-03 01:45:01 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [7]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24423656|             8|
|   24297214|             1|
|   12879980|             3|
|     515450|             2|
|   13313689|             1|
|   15523729|             4|
|    1673863|             1|
|   14552054|             2|
|   45392827|            14|
|   44178035|             1|
|   10522786|            10|
|   44848424|             3|
|   14230926|             1|
|   49243158|           274|
|   49084939|            10|
|   28777148|            41|
|   41836864|             1|
|    5219946|             1|
|   37795150|             1|
|   52081222|             1|
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001DE29SS|Chaplin (15th Ann...|
|B00ZJ2GL6G|Jurassic World 3D...|
|B00T5DM876|Pretty Little Lia...|
|B004VW4V76|         Lebanon, PA|
|B009D4SFEC|Murdoch Mysteries...|
|B000GG4Y5K|Numb3rs - The Com...|
|B00383XZP8|TCM Spotlight: Ch...|
|B00I9MS86O|Breaking Bad: The...|
|B00005K3O4|Killer Klowns Fro...|
|B00BD6KETC|     Psych: Season 7|
|B00WHZZBTQ|Heaven Adores You...|
|B000092Q5C|Remo Williams - T...|
|B004AXPWH4|10 MINUTESOLUTION...|
|B00FEP9PQG|Boardwalk Empire:...|
|B0002F6BJW|Fractured Flicker...|
|B0007LXP7W|The A-Team - Seas...|
|B005ET4NHI|  Where The Boys Are|
|B005EBOSO4|Muscle Energy Tec...|
|B003V5CTKU|Black Label Socie...|
|B000087F6L|      Fahrenheit 451|
+----------+--------------------+
only showing top 20 rows



In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cpyeyia3a8p5.us-east-2.rds.amazonaws.com:5432/Amazon_Reviews"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}


In [12]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


Py4JJavaError: ignored

In [13]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [14]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [15]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)